In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import pdal
from dask.distributed import Client

np.set_printoptions(precision=4, suppress=True)

from forest_structure_tools.metrics import forest_structure_metrics

In [2]:
data_dir = Path("../data")
plots_dir = data_dir / "outputs" / "plots"
plots_lidar_dir = plots_dir / "lidar"

plots_gdf = gpd.read_file(plots_dir / "plots.geojson")
plot_ids = plots_gdf['id'].to_list()

def read_plot_lidar(plot_id: str):
    pl = pdal.Reader(str(plots_lidar_dir / f"{plot_id}.copc.laz")).pipeline()
    pl.execute()

    return pl.arrays[0]

In [3]:
metrics_dir = plots_dir / "metrics"
metrics_dir.mkdir(parents=True, exist_ok=True)

In [4]:
def add_suffix(ds: xr.Dataset, suffix: str):
    new_names = {var: var + suffix for var in ds.data_vars}
    ds_suffixed = ds.rename(new_names)
    return ds_suffixed


def calculate_metrics(
    plot_id: id,
    xy_bin_size: float | None = None,
    z_bin_size: float | None = 1,
):
    points = read_plot_lidar(plot_id)
    x = points["X"]
    y = points["Y"]
    z = points["Z"]

    weights = 1 / points["NumberOfReturns"]

    unweighted_metrics = forest_structure_metrics(
        x=x,
        y=y,
        z=z,
        xy_bin_size=xy_bin_size,
        z_bin_size=z_bin_size,
        include_basic=True,
        cover_threshold=0,
        percentiles=np.arange(10, 100, 10),
        weights=None,
    )
    weighted_metrics = forest_structure_metrics(
        x=x,
        y=y,
        z=z,
        xy_bin_size=xy_bin_size,
        z_bin_size=z_bin_size,
        # Only need cover and veg profile metrics for weighted
        include_basic=False,
        percentiles=None,
        cover_threshold=None,
        weights=weights,
    ).pipe(lambda ds: add_suffix(ds, "[w]"))
    
    metrics = xr.merge([unweighted_metrics, weighted_metrics])


    metrics.attrs["xy_bin_size"] = str(xy_bin_size)
    metrics.attrs["z_bin_size"] = str(z_bin_size)

    return metrics

def calculate_and_save_metrics(plot_id : str, xy_bin_size: float | None = None, z_bin_size: float | None = 1):
    
    xy = f'{xy_bin_size}' if xy_bin_size is not None else 'none'
    z = f'{z_bin_size}' if z_bin_size is not None else 'none'

    sub_dir = metrics_dir / f'xy_{xy}_z_{z}'
    
    if sub_dir.exists() is False:
        sub_dir.mkdir(parents=True, exist_ok=True)

    metrics = calculate_metrics(plot_id, xy_bin_size=xy_bin_size, z_bin_size=z_bin_size)
    metrics.to_netcdf(sub_dir / f"{plot_id}.nc")




In [5]:
calculate_metrics("AGG_O_01_P1", xy_bin_size=10, z_bin_size=1)

<xarray.Dataset> Size: 165kB
Dimensions:             (x: 8, y: 8, z: 18)
Coordinates:
  * x                   (x) float64 64B 4.63e+05 4.63e+05 ... 4.63e+05 4.63e+05
  * y                   (y) float64 64B 5.26e+06 5.26e+06 ... 5.26e+06 5.26e+06
  * z                   (z) float64 144B 0.0 1.0 2.0 3.0 ... 14.0 15.0 16.0 17.0
Data variables: (12/49)
    max_veg             (x, y) float64 512B nan nan 12.95 ... 9.855 nan nan
    mean_veg            (x, y) float64 512B nan nan 5.282 ... 5.027 nan nan
    median_veg          (x, y) float64 512B nan nan 5.205 ... 5.446 nan nan
    sd_veg              (x, y) float64 512B nan nan 3.388 ... 2.903 nan nan
    var_veg             (x, y) float64 512B nan nan 11.48 ... 8.427 nan nan
    cv_veg              (x, y) float64 512B nan nan 0.6413 ... 0.5775 nan nan
    ...                  ...
    pad_z[w]            (z, x, y) float64 9kB nan nan nan nan ... nan nan nan
    fhd[w]              (x, y) float64 512B nan nan 2.371 ... 2.099 nan nan
    cv_pct_inside_z[w]  (x, y) float64 512B nan nan 0.7143 ... 0.7981 nan nan
    cv_lgap_z[w]        (x, y) float64 512B nan nan 0.3569 ... 0.106 nan nan
    cv_pai_z[w]         (x, y) float64 512B nan nan 2.231 ... 0.8792 nan nan
    cv_pad_z[w]         (x, y) float64 512B nan nan 2.231 ... 0.8792 nan nan
Attributes:
    xy_bin_size:  10
    z_bin_size:   1

In [6]:
calculate_and_save_metrics("AGG_O_01_P1", xy_bin_size=0.5, z_bin_size=None)

In [7]:
client = Client()  # Start a Dask client
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55594,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:55605,Total threads: 2
Dashboard: http://127.0.0.1:55607/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:55597,


In [8]:
# XY None
futures = client.map(calculate_and_save_metrics, plot_ids, xy_bin_size=None)
results = client.gather(futures)

In [9]:
# XY 10 m
futures = client.map(calculate_and_save_metrics, plot_ids, xy_bin_size=10)
results = client.gather(futures)

In [10]:
# XY 5 m
futures = client.map(calculate_and_save_metrics, plot_ids, xy_bin_size=5)
results = client.gather(futures)

In [11]:
# XY 1 m 
futures = client.map(calculate_and_save_metrics, plot_ids, xy_bin_size=1)
results = client.gather(futures)

In [12]:
# XY 0.5 m (no z)
futures = client.map(calculate_and_save_metrics, plot_ids, xy_bin_size=0.5, z_bin_size=None)
results = client.gather(futures)

In [13]:
# client.close()

In [14]:
# These snippets are useful for merging datasets.
# def add_plot_dimension_to_ds(ds: xr.Dataset, plot_id: str) -> xr.Dataset:
#     site = plot_id[:-3]
#     site_type = site[:3]

#     ds = ds.expand_dims(plot=[plot_id]).assign_coords(
#         plot=("plot", [plot_id]), site=("plot", [site]), site_type=("plot", [site_type])
#     )

#     return ds

# if xy_bin_size is not None:
#     x_offset = metrics.x.min().item()
#     y_offset = metrics.y.min().item()

#     metrics["x_offset"] = x_offset
#     metrics["y_offset"] = y_offset

#     metrics["x"] = metrics["x"] - x_offset
#     metrics["y"] = metrics["y"] - y_offset

# metrics = add_plot_dimension_to_ds(metrics, plot_id)